# Function to loop over sedative totals
- Erina Ghosh 17/04/18
- Loop over all encounters analyze bolus & IV separately

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import Sedative_daily_totals_etl as med_etl

In [2]:
med_analyze = med_etl.sedShiftTotal()

### Reading Med File

In [3]:
med_data = pd.read_csv('170415_Medlist4_vent_duration.csv', 
                       parse_dates=['starttime', 'endtime','STARTTIME', 'ENDTIME','intime'])
med_data.shape

(116116, 35)

In [4]:
filter_cohort = med_data['ICUSTAY_ID'].unique()
filter_cohort_list = list(filter_cohort)

In [5]:
def dn_totals(med_df):
    if type(med_df.index[0]) is not pd.tslib.Timestamp:
        med_df.set_index('Shift_Start', drop=False, inplace=True)
    col_list = med_df.columns
    med_day = med_df[med_df.index.hour == 7]
    med_night = med_df[med_df.index.hour == 19]
    med_totals = {}
    for col in col_list:
        if col != 'Shift_Start':
            key1 = 'D_'+ col
            med_totals[key1] = med_day[col].sum()
            key2 = 'N_'+col
            med_totals[key2] = med_night[col].sum()
    return med_totals

In [6]:
bolus_sum_dict = {}
iv_sum_dict = {}
for i in range(10):
    enc_id = filter_cohort_list[i]
    sub_med_data = med_data.loc[med_data['ICUSTAY_ID'] == enc_id]
    med_bolus_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '05-Med Bolus']
    med_iv_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '01-Drips']
    if med_bolus_df.shape[0] > 0:
        bolus_shift_totals = med_analyze.get_bolus_shift_total(med_bolus_df)
        totals_dict = dn_totals(bolus_shift_totals)
        bolus_sum_dict[enc_id] = totals_dict
    if med_iv_df.shape[0] > 0:
        iv_shift_totals = med_analyze.get_IV_shift_total(med_iv_df)
        if iv_shift_totals.shape[0] > 0:
            total_iv_dict = dn_totals(iv_shift_totals)
            iv_sum_dict[enc_id] = total_iv_dict
    print enc_id

//anaconda/envs/py27/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda/envs/py27/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
Sedative_daily_totals_etl.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pyd

263099.0
215903.0
260189.0
299133.0
241570.0
213042.0
253242.0
289598.0
250366.0
250989.0


In [7]:
iv_sum_dict

{213042.0: {'D_Propofol': 424.4464390438896, 'N_Propofol': 1515.7406506730913},
 215903.0: {'D_Propofol': 859.87798030359761,
  'N_Propofol': 1188.6116747715671},
 241570.0: {'D_Fentanyl': 3.4661188469608333,
  'D_Midazolam (Versed)': 172.21957924024997,
  'N_Fentanyl': 5.7243559691499994,
  'N_Midazolam (Versed)': 164.51260861583333},
 250366.0: {'D_Fentanyl (Concentrate)': 1.2780612548399999,
  'D_Midazolam (Versed)': 83.159085499000014,
  'D_Morphine Sulfate': nan,
  'D_Propofol': 825.49037970337963,
  'N_Fentanyl (Concentrate)': 1.6328741681299999,
  'N_Midazolam (Versed)': 63.19221862933334,
  'N_Morphine Sulfate': 15.6265835825,
  'N_Propofol': 279.48505318703531},
 250989.0: {'D_Fentanyl (Concentrate)': 0.27535903479499996,
  'D_Midazolam (Versed)': 4.4779474234666665,
  'D_Propofol': nan,
  'N_Fentanyl (Concentrate)': 0.55406297634000001,
  'N_Midazolam (Versed)': 8.222451671,
  'N_Propofol': 78.644459889956991},
 253242.0: {'D_Fentanyl (Concentrate)': 4.1618029291466669,
  'D_

In [10]:
iv_sum_df = pd.DataFrame.from_dict(iv_sum_dict, orient='index')

In [11]:
iv_sum_df.head()

,D_Midazolam (Versed),D_Fentanyl,N_Fentanyl,N_Midazolam (Versed),N_Fentanyl (Concentrate),D_Propofol,D_Fentanyl (Concentrate),N_Propofol,D_Morphine Sulfate,N_Morphine Sulfate,D_Hydromorphone (Dilaudid),N_Hydromorphone (Dilaudid)
213042.0,NaN,NaN,NaN,NaN,NaN,424.446439,NaN,1515.740651,NaN,NaN,NaN,NaN
215903.0,NaN,NaN,NaN,NaN,NaN,859.877980,NaN,1188.611675,NaN,NaN,NaN,NaN
241570.0,172.219579,3.466119,5.724356,164.512609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250366.0,83.159085,NaN,NaN,63.192219,1.632874,825.490380,1.278061,279.485053,NaN,15.626584,NaN,NaN
250989.0,4.477947,NaN,NaN,8.222452,0.554063,NaN,0.275359,78.644460,NaN,NaN,NaN,NaN


## Running Loop on all encounters

In [12]:
bolus_sum_dict = {}
iv_sum_dict = {}
for i in range(len(filter_cohort_list)):
    enc_id = filter_cohort_list[i]
    sub_med_data = med_data.loc[med_data['ICUSTAY_ID'] == enc_id]
    med_bolus_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '05-Med Bolus']
    med_iv_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '01-Drips']
    if med_bolus_df.shape[0] > 0:
        bolus_shift_totals = med_analyze.get_bolus_shift_total(med_bolus_df)
        totals_dict = dn_totals(bolus_shift_totals)
        bolus_sum_dict[enc_id] = totals_dict
    if med_iv_df.shape[0] > 0:
        iv_shift_totals = med_analyze.get_IV_shift_total(med_iv_df)
        if iv_shift_totals.shape[0] > 0:
            total_iv_dict = dn_totals(iv_shift_totals)
            iv_sum_dict[enc_id] = total_iv_dict
    if i%200 == 0:
        print enc_id

263099.0
247013.0
233221.0
216923.0
291872.0
247444.0
293347.0
267383.0
251345.0
254524.0
220107.0
265005.0


In [13]:
iv_sum_df = pd.DataFrame.from_dict(iv_sum_dict, orient='index')

In [14]:
bolus_sum_df = pd.DataFrame.from_dict(bolus_sum_dict, orient='index')

In [15]:
iv_sum_df.shape

(2190, 18)

In [16]:
bolus_sum_df.shape

(2131, 22)

In [17]:
iv_sum_df.tail()

,N_Fentanyl (Concentrate),D_Propofol,D_Fentanyl (Concentrate),N_Propofol,D_Midazolam (Versed),N_Midazolam (Versed),D_Fentanyl,N_Fentanyl,D_Dexmedetomidine (Precedex),N_Dexmedetomidine (Precedex),D_Morphine Sulfate,N_Morphine Sulfate,N_Lorazepam (Ativan),D_Lorazepam (Ativan),D_Methadone Hydrochloride,N_Methadone Hydrochloride,D_Hydromorphone (Dilaudid),N_Hydromorphone (Dilaudid)
299833.0,NaN,104.258442,NaN,64.427312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299846.0,NaN,380.757721,NaN,62.026991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299847.0,NaN,277.835109,NaN,258.626454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299866.0,NaN,326.428899,NaN,20.713355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299872.0,NaN,NaN,NaN,NaN,5.765655,16.671166,0.65326,0.547891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
bolus_sum_df.tail()

,D_Hydromorphone (Dilaudid),D_Lorazepam (Ativan),D_Propofol,D_Midazolam (Versed),N_Propofol,D_Fentanyl,N_Fentanyl,N_Lorazepam (Ativan),N_Hydromorphone (Dilaudid),N_Midazolam (Versed),...,D_Meperidine (Demerol),N_Meperidine (Demerol),D_Diazepam (Valium),N_Diazepam (Valium),N_Vecuronium,D_Vecuronium,D_Methadone Hydrochloride,N_Methadone Hydrochloride,N_Cisatracurium,D_Cisatracurium
299716.0,14.750001,2.0,NaN,NaN,NaN,NaN,NaN,NaN,13.500001,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299833.0,NaN,NaN,NaN,NaN,NaN,0.55,0.4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299846.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299847.0,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.400000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299866.0,4.000000,NaN,10.000001,NaN,NaN,NaN,NaN,NaN,3.750000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Saving files

In [19]:
iv_sum_df.to_csv('170418_IV_DN_totals.csv')
bolus_sum_df.to_csv('170418_bolus_DN_totals.csv')

In [20]:
bolus_sum_df.describe()

,D_Hydromorphone (Dilaudid),D_Lorazepam (Ativan),D_Propofol,D_Midazolam (Versed),N_Propofol,D_Fentanyl,N_Fentanyl,N_Lorazepam (Ativan),N_Hydromorphone (Dilaudid),N_Midazolam (Versed),...,D_Meperidine (Demerol),N_Meperidine (Demerol),D_Diazepam (Valium),N_Diazepam (Valium),N_Vecuronium,D_Vecuronium,D_Methadone Hydrochloride,N_Methadone Hydrochloride,N_Cisatracurium,D_Cisatracurium
count,438.000000,320.000000,559.000000,674.000000,499.000000,1108.000000,1077.000000,339.000000,426.000000,640.000000,...,45.000000,50.00000,55.000000,56.000000,18.000000,46.000000,21.000000,17.000000,0.0,4.000000
mean,4.201735,3.904531,62.214608,6.918744,55.066237,0.359223,0.359351,4.268668,4.629319,7.861953,...,14.316667,17.54400,37.090912,40.613099,14.805557,14.815218,70.333338,103.058830,NaN,9.625001
std,12.616014,7.137792,84.625900,10.515779,66.801745,0.498997,0.507903,7.302154,15.418732,11.940019,...,9.649600,11.87337,57.918696,44.763062,14.032671,20.193332,90.682602,126.489864,NaN,3.497022
min,0.000000,-10.000000,-120.000012,-35.000002,-10.000001,-0.375000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,3.500000,0.000000,5.000000,2.000000,NaN,5.000000
25%,0.750000,1.000000,20.000002,2.000000,10.000001,0.075000,0.075000,0.875000,0.750000,2.000000,...,12.500000,12.50000,10.000001,10.000001,7.750000,8.500001,10.000001,20.000001,NaN,8.750001
50%,1.500000,2.000000,40.000003,3.500000,30.000003,0.200000,0.200000,2.000000,1.900000,4.000000,...,12.500000,12.50000,15.000001,20.000002,10.000001,10.000001,30.000001,55.000001,NaN,10.000001
75%,4.000000,4.000000,80.000006,8.000000,70.000007,0.425000,0.450000,4.000000,4.500000,9.000000,...,25.000000,25.00000,43.750004,54.375004,16.000001,10.000001,100.000005,140.000011,NaN,10.875001
max,234.000012,80.000005,1235.000055,101.500005,680.000043,4.322500,5.175000,61.333336,289.000014,125.000007,...,37.500000,62.50000,385.000020,195.000019,60.000002,100.000000,320.000016,420.000025,NaN,13.500000


In [31]:
iv_sum_stats = iv_sum_df.describe()

In [32]:
# Saving stats file
iv_sum_stats.to_csv('170418_IV_DN_Stats.csv')

### Counting negative values in boluses

In [26]:
bolus_sum_df[bolus_sum_df['D_Midazolam (Versed)'] < 0]

,D_Hydromorphone (Dilaudid),D_Lorazepam (Ativan),D_Propofol,D_Midazolam (Versed),N_Propofol,D_Fentanyl,N_Fentanyl,N_Lorazepam (Ativan),N_Hydromorphone (Dilaudid),N_Midazolam (Versed),...,D_Meperidine (Demerol),N_Meperidine (Demerol),D_Diazepam (Valium),N_Diazepam (Valium),N_Vecuronium,D_Vecuronium,D_Methadone Hydrochloride,N_Methadone Hydrochloride,N_Cisatracurium,D_Cisatracurium
205250.0,NaN,NaN,NaN,-26.000001,30.000003,-0.375,1.275,NaN,NaN,31.500001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235675.0,NaN,NaN,NaN,-35.000002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
neg_val = bolus_sum_df[bolus_sum_df <0]

In [24]:
neg_val.shape

(2131, 22)

,D_Hydromorphone (Dilaudid),D_Lorazepam (Ativan),D_Propofol,D_Midazolam (Versed),N_Propofol,D_Fentanyl,N_Fentanyl,N_Lorazepam (Ativan),N_Hydromorphone (Dilaudid),N_Midazolam (Versed),...,D_Meperidine (Demerol),N_Meperidine (Demerol),D_Diazepam (Valium),N_Diazepam (Valium),N_Vecuronium,D_Vecuronium,D_Methadone Hydrochloride,N_Methadone Hydrochloride,N_Cisatracurium,D_Cisatracurium
200063.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200069.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200087.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200339.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Count rows with negative values

In [27]:
neg_rows = med_data.loc[med_data['AMOUNT'] < 0]
neg_rows.shape

(220, 35)

In [29]:
neg_rows[['ICUSTAY_ID','STARTTIME', 'ENDTIME', 'intime', 'starttime', 'endtime', 'LABEL', 'AMOUNT', 'AMOUNTUOM']].head()

,ICUSTAY_ID,STARTTIME,ENDTIME,intime,starttime,endtime,LABEL,AMOUNT,AMOUNTUOM
822,212644.0,2195-11-11 16:04:00,2195-11-11 09:13:00,2195-11-08 23:10:54,2195-11-09 00:00:00,2195-11-12 16:00:00,Midazolam (Versed),-13.700000,mg
1745,235675.0,2174-04-13 12:10:00,2174-04-13 11:49:00,2174-04-12 11:00:25,2174-04-12 15:00:00,2174-04-17 08:00:00,Midazolam (Versed),-42.000002,mg
2798,264104.0,2116-11-23 22:55:00,2116-11-23 22:47:00,2116-11-20 20:08:54,2116-11-21 07:00:00,2116-12-01 10:26:00,Propofol,-20.573793,mg
3311,207857.0,2137-05-12 03:31:00,2137-05-12 03:08:00,2137-05-07 02:07:24,2137-05-07 02:00:00,2137-05-24 16:00:00,Propofol,-83.282075,mg
3604,253903.0,2116-06-28 12:37:00,2116-06-28 12:16:00,2116-06-27 21:59:15,2116-06-28 08:24:00,2116-06-30 13:00:00,Propofol,-106.315942,mg


In [33]:
neg_rows.groupby(['ORDERCATEGORYNAME']).count()

,Unnamed: 0,hadm_id,starttime,endtime,age,dayHrs,nightHrs,intime,los,icustay_id,...,ORDERCOMPONENTTYPEDESCRIPTION,ORDERCATEGORYDESCRIPTION,PATIENTWEIGHT,TOTALAMOUNT,TOTALAMOUNTUOM,STATUSDESCRIPTION,sed_flag,LABEL,nmb_flag,In_timeframe
ORDERCATEGORYNAME,,,,,,,,,,,,,,,,,,,,,
01-Drips,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200
05-Med Bolus,20,20,20,20,20,20,20,20,20,20,...,20,20,20,0,0,20,20,20,20,20


In [38]:
neg_rows.loc[neg_rows['ORDERCATEGORYNAME'] == '01-Drips'].iloc[0]

Unnamed: 0                                           1071
hadm_id                                            172420
starttime                             2195-11-09 00:00:00
endtime                               2195-11-12 16:00:00
age                                                    66
dayHrs                                                 45
nightHrs                                               43
intime                                2195-11-08 23:10:54
los                                                 4.254
icustay_id                                         212644
ROW_ID                                             197525
SUBJECT_ID                                           1459
HADM_ID                                            172420
ICUSTAY_ID                                         212644
STARTTIME                             2195-11-11 16:04:00
ENDTIME                               2195-11-11 09:13:00
ITEMID                                             221668
AMOUNT        

## Running Loop for bolus + IV
- Removing all rows with negative values (AMOUNT column)

In [39]:
pos_med_data = med_data.loc[med_data['AMOUNT'] >= 0]
pos_med_data.shape

(115896, 35)

In [40]:
bolus_sum_dict = {}
iv_sum_dict = {}
for i in range(len(filter_cohort_list)):
    enc_id = filter_cohort_list[i]
    sub_med_data = pos_med_data.loc[pos_med_data['ICUSTAY_ID'] == enc_id]
    med_bolus_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '05-Med Bolus']
    med_iv_df = sub_med_data.loc[sub_med_data['ORDERCATEGORYNAME'] == '01-Drips']
    if med_bolus_df.shape[0] > 0:
        bolus_shift_totals = med_analyze.get_bolus_shift_total(med_bolus_df)
        totals_dict = dn_totals(bolus_shift_totals)
        bolus_sum_dict[enc_id] = totals_dict
    if med_iv_df.shape[0] > 0:
        iv_shift_totals = med_analyze.get_IV_shift_total(med_iv_df)
        if iv_shift_totals.shape[0] > 0:
            total_iv_dict = dn_totals(iv_shift_totals)
            iv_sum_dict[enc_id] = total_iv_dict
    if i%200 == 0:
        print enc_id

263099.0
247013.0
233221.0
216923.0
291872.0
247444.0
293347.0
267383.0
251345.0
254524.0
220107.0
265005.0


In [41]:
iv_sum_df = pd.DataFrame.from_dict(iv_sum_dict, orient='index')
bolus_sum_df = pd.DataFrame.from_dict(bolus_sum_dict, orient='index')

In [42]:
print iv_sum_df.shape, bolus_sum_df.shape

(2190, 18) (2131, 22)


In [43]:
iv_sum_stats = iv_sum_df.describe()
iv_sum_stats

,N_Fentanyl (Concentrate),D_Propofol,D_Fentanyl (Concentrate),N_Propofol,D_Midazolam (Versed),N_Midazolam (Versed),D_Fentanyl,N_Fentanyl,D_Dexmedetomidine (Precedex),N_Dexmedetomidine (Precedex),D_Morphine Sulfate,N_Morphine Sulfate,N_Lorazepam (Ativan),D_Lorazepam (Ativan),D_Methadone Hydrochloride,N_Methadone Hydrochloride,D_Hydromorphone (Dilaudid),N_Hydromorphone (Dilaudid)
count,699.000000,1807.000000,703.000000,1844.000000,723.000000,724.000000,299.000000,301.000000,254.000000,234.000000,35.000000,28.000000,5.000000,5.000000,2.000000,2.000000,4.000000,4.000000
mean,2.117775,695.331369,1.855714,694.108679,56.863673,63.277053,1.837837,2.097265,0.109567,0.114540,23.042194,32.037681,31.372064,32.310823,46.583336,26.833335,6.842066,11.324654
std,2.916046,836.549761,2.716401,909.906087,93.998477,100.685538,2.003096,2.305259,0.121130,0.145986,30.408106,26.825044,22.705370,16.964772,47.258305,18.620479,6.643620,9.475030
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009167,0.000000,0.000000,0.000000,1.066667,4.776771,15.965082,13.166668,13.666668,1.790389,1.300069
25%,0.538554,196.904091,0.415618,146.738407,8.980793,12.000000,0.600666,0.645000,0.030255,0.028625,3.958333,15.363310,15.543137,16.164403,29.875002,20.250001,2.626664,4.351382
50%,1.254781,424.446439,1.011667,402.334527,29.530847,33.629169,1.200000,1.250276,0.068160,0.077638,10.505618,25.300063,27.605690,32.245909,46.583336,26.833335,4.595911,11.500027
75%,2.399969,859.248007,2.149483,886.711493,61.613893,71.122573,2.280919,2.703381,0.150737,0.139608,33.458333,46.625000,51.054163,41.757887,63.291670,33.416668,8.811313,18.473300
max,26.985959,8402.097150,26.841766,9462.952952,1169.588810,1203.906544,11.970689,12.805895,0.929020,1.078815,153.991948,121.206434,57.880556,55.420834,80.000004,40.000002,16.386053,20.998494


In [44]:
bolus_sum_stats = bolus_sum_df.describe()
bolus_sum_stats

,D_Hydromorphone (Dilaudid),D_Lorazepam (Ativan),D_Propofol,D_Midazolam (Versed),N_Propofol,D_Fentanyl,N_Fentanyl,N_Lorazepam (Ativan),N_Hydromorphone (Dilaudid),N_Midazolam (Versed),...,D_Meperidine (Demerol),N_Meperidine (Demerol),D_Diazepam (Valium),N_Diazepam (Valium),N_Vecuronium,D_Vecuronium,D_Methadone Hydrochloride,N_Methadone Hydrochloride,N_Cisatracurium,D_Cisatracurium
count,438.000000,320.000000,558.000000,674.000000,498.000000,1108.000000,1077.000000,339.000000,426.000000,640.000000,...,45.000000,50.00000,55.000000,56.000000,18.000000,46.000000,21.000000,17.000000,0.0,4.000000
mean,4.201735,3.957656,62.702448,7.084916,55.196892,0.361874,0.361718,4.268668,4.634601,7.882657,...,14.316667,17.54400,37.090912,40.613099,14.805557,14.815218,70.333338,103.058830,NaN,9.625001
std,12.616014,7.097127,84.506247,10.399523,66.805064,0.505318,0.512284,7.302154,15.426796,12.021498,...,9.649600,11.87337,57.918696,44.763062,14.032671,20.193332,90.682602,126.489864,NaN,3.497022
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,3.500000,0.000000,5.000000,2.000000,NaN,5.000000
25%,0.750000,1.000000,20.000002,2.000000,10.000001,0.075000,0.075000,0.875000,0.750000,2.000000,...,12.500000,12.50000,10.000001,10.000001,7.750000,8.500001,10.000001,20.000001,NaN,8.750001
50%,1.500000,2.000000,40.000003,3.500000,30.000003,0.200000,0.200000,2.000000,1.900000,4.000000,...,12.500000,12.50000,15.000001,20.000002,10.000001,10.000001,30.000001,55.000001,NaN,10.000001
75%,4.000000,4.000000,80.000006,8.000000,70.000007,0.425000,0.450000,4.000000,4.500000,9.000000,...,25.000000,25.00000,43.750004,54.375004,16.000001,10.000001,100.000005,140.000011,NaN,10.875001
max,234.000012,80.000005,1235.000055,101.500005,680.000043,4.322500,5.175000,61.333336,289.000014,125.000007,...,37.500000,62.50000,385.000020,195.000019,60.000002,100.000000,320.000016,420.000025,NaN,13.500000


In [45]:
# Saving files
iv_sum_df.to_csv('170419_IV_DN_totals.csv')
bolus_sum_df.to_csv('170419_bolus_DN_totals.csv')
iv_sum_stats.to_csv('170419_IV_DN_stats.csv')
bolus_sum_stats.to_csv('170419_bolus_DN_stats.csv')